In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import string
import os
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Download required NLTK data
nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

class ExcelQuestionAnswerer:
    def __init__(self, file_path):
        """
        Initialize the question answerer with an Excel file.

        Args:
        f   ile_path (str): Path to the Excel file
        """
        self.file_path = file_path
        self.df = None
        self.column_descriptions = {}
        self.data_summary = {}
        self.stop_words = set(stopwords.words('english'))
        self.lemmatizer = WordNetLemmatizer()
        self.question_classifier = None

        # Load data and continue only if successful
        if self.load_data():
            self.analyze_data()
            self.generate_qa_templates()
        else:
            print("Failed to initialize the Excel Question Answerer. Please check the file path.")
            # Initialize empty structures to prevent errors
            self.data_summary = {
                'total_rows': 0,
                'total_columns': 0,
                'column_names': [],
                'column_types': {}
            }
            self.qa_templates = {}
            self.example_questions = []

    def load_data(self):
        """Load the Excel file into a pandas DataFrame."""
        if not os.path.exists(self.file_path):
            print(f"Excel file not found: {self.file_path}")
            print("Please ensure the file exists at the specified path.")
            self.df = None
            return False  # Return False instead of exiting

        try:
            self.df = pd.read_excel(self.file_path)
            print(f"Successfully loaded data from {self.file_path}")
            print(f"Shape: {self.df.shape}")
            return True  # Return True on success
        except Exception as e:
            print(f"Error loading Excel file: {e}")
            self.df = None
            return False  # Return False on error

    def analyze_data(self):
        """Analyze the data to generate insights."""
        # Check if DataFrame is loaded properly
        if self.df is None:
            print("No data to analyze. Please check if the Excel file was loaded correctly.")
            # Initialize empty dictionaries to prevent further errors
            self.data_summary = {
                'total_rows': 0,
                'total_columns': 0,
                'column_names': [],
                'column_types': {}
            }
            return
        self.data_summary['total_rows'] = len(self.df)
        self.data_summary['total_columns'] = len(self.df.columns)
        self.data_summary['column_names'] = list(self.df.columns)
        self.data_summary['column_types'] = self.df.dtypes.to_dict()

        # For each column, gather descriptive statistics
        for column in self.df.columns:
            self.column_descriptions[column] = {}

            # Get column data type
            col_type = self.df[column].dtype

            if pd.api.types.is_numeric_dtype(col_type):
                # Numeric column
                self.column_descriptions[column]['type'] = 'numeric'
                self.column_descriptions[column]['min'] = self.df[column].min()
                self.column_descriptions[column]['max'] = self.df[column].max()
                self.column_descriptions[column]['mean'] = self.df[column].mean()
                self.column_descriptions[column]['median'] = self.df[column].median()
                self.column_descriptions[column]['std'] = self.df[column].std()
                self.column_descriptions[column]['missing'] = self.df[column].isna().sum()

            elif pd.api.types.is_datetime64_dtype(col_type):
                # Datetime column
                self.column_descriptions[column]['type'] = 'datetime'
                self.column_descriptions[column]['min'] = self.df[column].min()
                self.column_descriptions[column]['max'] = self.df[column].max()
                self.column_descriptions[column]['range_days'] = (self.df[column].max() - self.df[column].min()).days
                self.column_descriptions[column]['missing'] = self.df[column].isna().sum()

            elif pd.api.types.is_string_dtype(col_type) or pd.api.types.is_object_dtype(col_type):
                # String/categorical column
                self.column_descriptions[column]['type'] = 'categorical'
                self.column_descriptions[column]['unique_values'] = self.df[column].nunique()
                self.column_descriptions[column]['most_common'] = self.df[column].value_counts().nlargest(5).to_dict()
                self.column_descriptions[column]['missing'] = self.df[column].isna().sum()

            # Try to infer column meaning from name
            if 'date' in column.lower() or 'time' in column.lower():
                self.column_descriptions[column]['semantic_type'] = 'date/time'
            elif 'price' in column.lower() or 'cost' in column.lower() or 'revenue' in column.lower():
                self.column_descriptions[column]['semantic_type'] = 'monetary'
            elif 'id' in column.lower() or 'key' in column.lower():
                self.column_descriptions[column]['semantic_type'] = 'identifier'
            elif 'name' in column.lower():
                self.column_descriptions[column]['semantic_type'] = 'name'
            elif 'qty' in column.lower() or 'quantity' in column.lower() or 'count' in column.lower():
                self.column_descriptions[column]['semantic_type'] = 'quantity'
            else:
                self.column_descriptions[column]['semantic_type'] = 'unknown'

        # Generate correlations for numeric columns
        numeric_columns = self.df.select_dtypes(include=['number']).columns
        if len(numeric_columns) >= 2:
            self.data_summary['correlations'] = self.df[numeric_columns].corr().to_dict()

    def generate_qa_templates(self):
        """Generate question and answer templates based on the data structure."""
        self.qa_templates = {
            "count": "How many {rows/column} are in the {dataset/column}?",
            "min": "What is the minimum value of {column}?",
            "max": "What is the maximum value of {column}?",
            "mean": "What is the average/mean value of {column}?",
            "sum": "What is the total/sum of {column}?",
            "unique": "What are the unique values in {column}?",
            "missing": "How many missing values are there in {column}?",
            "correlation": "Is there a correlation between {column1} and {column2}?",
            "filter": "Show me all rows where {column} {condition}",
            "group": "What is the {aggregation} of {column} grouped by {group_column}?",
            "top": "What are the top {N} values in {column}?",
            "bottom": "What are the bottom {N} values in {column}?",
            "distribution": "What is the distribution of {column}?",
            "trend": "What is the trend of {column} over {time_column}?"
        }

        # Generate example questions for training
        self.example_questions = self.generate_example_questions()
    def interactive_qa(self):
        """Start an interactive Q&A session with the user."""
        # Check if the data was loaded successfully
        if self.df is None:
           print("Cannot start interactive session: No data was loaded.")
           return

        # Train the question classifier if not already trained
        if self.question_classifier is None:
            print("Training question classifier...")
            self.train_question_classifier()

        print("\n===== Excel Question Answering System =====")
        print(f"Loaded file: {self.file_path}")
        print(f"Dataset has {self.data_summary['total_rows']} rows and {self.data_summary['total_columns']} columns")
        print("Columns:", ", ".join(self.data_summary['column_names']))
        print("\nAsk questions about your data (type 'exit' to quit):")

        while True:
            question = input("\nYour question: ")

            if question.lower() in ['exit', 'quit', 'bye']:
               print("Goodbye!")
               break

            # Process the question and get the answer
            answer = self.answer_question(question)

            # Print the answer
            print("\nAnswer:")
            print(answer)

    def generate_example_questions(self):
        """Generate example questions for each template using actual column names."""
        examples = []

        # Get column names by type for appropriate substitution
        numeric_cols = self.df.select_dtypes(include=['number']).columns.tolist()
        categorical_cols = [col for col in self.df.columns if self.df[col].nunique() < 20 and not pd.api.types.is_numeric_dtype(self.df[col].dtype)]
        datetime_cols = [col for col, desc in self.column_descriptions.items() if desc.get('semantic_type') == 'date/time']

        # For each template, generate at least 3 example questions
        if numeric_cols:
            # Count questions
            examples.append(("How many rows are in the dataset?", "count"))
            examples.append((f"How many records does this data have?", "count"))
            examples.append((f"What's the total number of entries?", "count"))

            # Min questions
            for col in numeric_cols[:3]:  # Use up to 3
                examples.append((f"What is the minimum value of {col}?", "min"))
                examples.append((f"What's the lowest {col}?", "min"))
                examples.append((f"What's the smallest value in the {col} column?", "min"))

            # Max questions
            for col in numeric_cols[:3]:
                examples.append((f"What is the maximum value of {col}?", "max"))
                examples.append((f"What's the highest {col}?", "max"))
                examples.append((f"What's the largest value in the {col} column?", "max"))

            # Mean questions
            for col in numeric_cols[:3]:
                examples.append((f"What is the average value of {col}?", "mean"))
                examples.append((f"What's the mean {col}?", "mean"))
                examples.append((f"Calculate the average {col}.", "mean"))

            # Sum questions
            for col in numeric_cols[:3]:
                examples.append((f"What is the total of {col}?", "sum"))
                examples.append((f"Sum all values in {col}.", "sum"))
                examples.append((f"What's the sum of {col}?", "sum"))

            # Correlation questions
            if len(numeric_cols) >= 2:
                for i in range(min(3, len(numeric_cols))):
                    for j in range(i+1, min(i+3, len(numeric_cols))):
                        examples.append((f"Is there a correlation between {numeric_cols[i]} and {numeric_cols[j]}?", "correlation"))
                        examples.append((f"How do {numeric_cols[i]} and {numeric_cols[j]} relate?", "correlation"))
                        examples.append((f"Does {numeric_cols[i]} correlate with {numeric_cols[j]}?", "correlation"))

            # Filter questions
            for col in numeric_cols[:3]:
                examples.append((f"Show me all rows where {col} > {self.df[col].mean():.1f}", "filter"))
                examples.append((f"Filter data where {col} is less than {self.df[col].min() + (self.df[col].max() - self.df[col].min())/4:.1f}", "filter"))
                examples.append((f"Which records have {col} equal to {self.df[col].median():.1f}?", "filter"))

            # Top/Bottom questions
            for col in numeric_cols[:3]:
                examples.append((f"What are the top 5 values in {col}?", "top"))
                examples.append((f"Show me the 10 highest {col}.", "top"))
                examples.append((f"What are the 3 largest values of {col}?", "top"))
                examples.append((f"What are the bottom 5 values in {col}?", "bottom"))
                examples.append((f"Show me the 10 lowest {col}.", "bottom"))
                examples.append((f"What are the 3 smallest values of {col}?", "bottom"))

            # Distribution questions
            for col in numeric_cols[:3]:
                examples.append((f"What is the distribution of {col}?", "distribution"))
                examples.append((f"Show me the spread of {col}.", "distribution"))
                examples.append((f"How is {col} distributed?", "distribution"))

        if categorical_cols:
            # Unique questions
            for col in categorical_cols[:3]:
                examples.append((f"What are the unique values in {col}?", "unique"))
                examples.append((f"What distinct values does {col} have?", "unique"))
                examples.append((f"List all possible values of {col}.", "unique"))

            # Group questions
            if numeric_cols and categorical_cols:
                for cat_col in categorical_cols[:2]:
                    for num_col in numeric_cols[:2]:
                        examples.append((f"What is the average of {num_col} grouped by {cat_col}?", "group"))
                        examples.append((f"Show me the sum of {num_col} for each {cat_col}.", "group"))
                        examples.append((f"Calculate the total {num_col} by {cat_col}.", "group"))

        if datetime_cols and numeric_cols:
            # Trend questions
            for time_col in datetime_cols[:2]:
                for value_col in numeric_cols[:2]:
                    examples.append((f"What is the trend of {value_col} over {time_col}?", "trend"))
                    examples.append((f"How does {value_col} change over {time_col}?", "trend"))
                    examples.append((f"Show me {value_col} over time using {time_col}.", "trend"))

        return examples

    def preprocess_question(self, question):
        """Preprocess the question for better matching."""
        # Convert to lowercase
        question = question.lower()

        # Remove punctuation
        question = question.translate(str.maketrans('', '', string.punctuation))

        # Tokenize
        tokens = word_tokenize(question)

        # Remove stop words
        filtered_tokens = [word for word in tokens if word not in self.stop_words]

        # Lemmatize words
        lemmatized_tokens = [self.lemmatizer.lemmatize(word) for word in filtered_tokens]

        return ' '.join(lemmatized_tokens)

    def extract_features(self, questions):
        """Extract features from a list of questions using TF-IDF."""
        # Preprocess questions
        processed_questions = [self.preprocess_question(q) for q in questions]

        # Create TF-IDF vectorizer
        vectorizer = TfidfVectorizer(max_features=100)

        # Fit and transform
        features = vectorizer.fit_transform(processed_questions)

        # Store vectorizer for future use
        self.vectorizer = vectorizer

        return features

    def train_question_classifier(self):
        """Train a classifier to identify question types."""
        if not self.example_questions:
            print("No example questions available for training. Generate examples first.")
            return False

        # Extract questions and labels
        questions, labels = zip(*self.example_questions)

        # Extract features
        X = self.extract_features(questions)
        y = labels

        # Split data
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Train classifier
        clf = RandomForestClassifier(n_estimators=100, random_state=42)
        clf.fit(X_train, y_train)

        # Evaluate
        y_pred = clf.predict(X_test)
        report = classification_report(y_test, y_pred)
        print("Classification Report:")
        print(report)

        # Store classifier
        self.question_classifier = clf

        return True

    def identify_question_type(self, question):
        """Identify the type of question being asked."""
        # If we have a trained classifier, use it
        if self.question_classifier is not None:
            # Preprocess and extract features
            processed_question = self.preprocess_question(question)
            question_vector = self.vectorizer.transform([processed_question])

            # Predict question type
            question_type = self.question_classifier.predict(question_vector)[0]

            # Calculate confidence score
            confidence_scores = self.question_classifier.predict_proba(question_vector)[0]
            max_score_idx = np.argmax(confidence_scores)
            confidence = confidence_scores[max_score_idx]

            # Fall back to similarity method if confidence is low
            if confidence < 0.6:
                return self.identify_question_type_by_similarity(question)

            # Extract relevant columns mentioned in the question
            mentioned_columns = []
            for column in self.df.columns:
                if column.lower() in question.lower():
                    mentioned_columns.append(column)

            # Try to identify conditions in filter-type questions
            conditions = None
            if "where" in question.lower() or "when" in question.lower():
                # Simple condition extraction, can be enhanced further
                condition_patterns = [
                    r"(greater|more|higher|larger|bigger) than (\d+\.?\d*)",
                    r"(less|lower|smaller) than (\d+\.?\d*)",
                    r"equals? (?:to )?(\d+\.?\d*)",
                    r"= ?(\d+\.?\d*)",
                    r"> ?(\d+\.?\d*)",
                    r"< ?(\d+\.?\d*)",
                    r"between (\d+\.?\d*) and (\d+\.?\d*)"
                ]

                for pattern in condition_patterns:
                    matches = re.findall(pattern, question.lower())
                    if matches:
                        conditions = matches
                        break

            # Return the identified question type and parameters
            return {
                'type': question_type,
                'confidence': confidence,
                'mentioned_columns': mentioned_columns,
                'conditions': conditions,
                'original_question': question,
                'processed_question': processed_question
            }
        else:
            # Fall back to similarity method if no classifier is trained
            return self.identify_question_type_by_similarity(question)

    def identify_question_type_by_similarity(self, question):
        """Identify question type using similarity to templates."""
        # Preprocess the question
        processed_question = self.preprocess_question(question)

        # Convert templates to processed form for comparison
        processed_templates = {k: self.preprocess_question(v) for k, v in self.qa_templates.items()}

        # Calculate similarity between question and templates
        vectorizer = TfidfVectorizer()

        # Create a combined list of the processed question and all processed templates
        all_texts = [processed_question] + list(processed_templates.values())

        # Fit the vectorizer to all texts
        tfidf_matrix = vectorizer.fit_transform(all_texts)

        # Calculate similarity between the question and each template
        # Complete the identify_question_type_by_similarity method
    def identify_question_type_by_similarity(self, question):
        """Identify question type using similarity to templates."""
        # Preprocess the question
        processed_question = self.preprocess_question(question)

        # Convert templates to processed form for comparison
        processed_templates = {k: self.preprocess_question(v) for k, v in self.qa_templates.items()}

        # Calculate similarity between question and templates
        vectorizer = TfidfVectorizer()

        # Create a combined list of the processed question and all processed templates
        all_texts = [processed_question] + list(processed_templates.values())

        # Fit the vectorizer to all texts
        tfidf_matrix = vectorizer.fit_transform(all_texts)

        # Calculate similarity between the question and each template
        similarities = {}
        for i, (q_type, _) in enumerate(processed_templates.items()):
            # Add 1 to i because the processed question is at index 0
            similarity = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[i+1:i+2])[0][0]
            similarities[q_type] = similarity

        # Find the most similar template
        question_type = max(similarities, key=similarities.get)
        confidence = similarities[question_type]

        # Extract mentioned columns
        mentioned_columns = []
        for column in self.df.columns:
            if column.lower() in question.lower():
                mentioned_columns.append(column)

        # Try to identify numeric values for comparisons
        numbers = re.findall(r'\b\d+\.?\d*\b', question)

        # Return the identified question type and parameters
        return {
            'type': question_type,
            'confidence': confidence,
            'mentioned_columns': mentioned_columns,
            'numbers': numbers if numbers else None,
            'original_question': question,
            'processed_question': processed_question
        }

    def answer_question(self, question):
        """Process a question and return the answer based on the data."""
        # First, identify the question type
        question_info = self.identify_question_type(question)

        # Get the column(s) mentioned in the question
        columns = question_info.get('mentioned_columns', [])

        # If no columns were explicitly mentioned but the question requires a column,
        # try to infer a relevant column based on the question type
        if not columns and question_info['type'] not in ['count']:
            # For questions about trends, prefer datetime columns
            if question_info['type'] == 'trend':
                for col, desc in self.column_descriptions.items():
                    if desc.get('semantic_type') == 'date/time':
                        columns.append(col)
                        break
            # For questions about averages, sums, etc., prefer numeric columns
            elif question_info['type'] in ['min', 'max', 'mean', 'sum', 'top', 'bottom', 'distribution']:
                for col, desc in self.column_descriptions.items():
                    if desc.get('type') == 'numeric':
                        columns.append(col)
                        break

        # Process the question based on its type
        try:
            if question_info['type'] == 'count':
                if 'column' in question.lower() and columns:
                    # Count unique values in the specified column
                    unique_count = self.df[columns[0]].nunique()
                    answer = f"There are {unique_count} unique values in the '{columns[0]}' column."
                else:
                    # Count rows in the dataset
                    answer = f"There are {self.data_summary['total_rows']} rows in the dataset."

            elif question_info['type'] == 'min' and columns:
                # Find minimum value
                min_val = self.df[columns[0]].min()
                answer = f"The minimum value of '{columns[0]}' is {min_val}."

            elif question_info['type'] == 'max' and columns:
                # Find maximum value
                max_val = self.df[columns[0]].max()
                answer = f"The maximum value of '{columns[0]}' is {max_val}."

            elif question_info['type'] == 'mean' and columns:
                # Calculate mean
                mean_val = self.df[columns[0]].mean()
                answer = f"The average/mean value of '{columns[0]}' is {mean_val:.2f}."

            elif question_info['type'] == 'sum' and columns:
                # Calculate sum
                sum_val = self.df[columns[0]].sum()
                answer = f"The sum of '{columns[0]}' is {sum_val}."

            elif question_info['type'] == 'unique' and columns:
                # Get unique values
                unique_vals = self.df[columns[0]].unique()
                if len(unique_vals) > 10:
                    # If there are many unique values, just show the count
                    answer = f"There are {len(unique_vals)} unique values in '{columns[0]}'."
                else:
                    # Otherwise, list all unique values
                    answer = f"The unique values in '{columns[0]}' are: {', '.join(map(str, unique_vals))}."

            elif question_info['type'] == 'missing' and columns:
                # Count missing values
                missing_count = self.df[columns[0]].isna().sum()
                answer = f"There are {missing_count} missing values in the '{columns[0]}' column."

            elif question_info['type'] == 'correlation' and len(columns) >= 2:
                # Calculate correlation between two columns
                corr = self.df[columns[0]].corr(self.df[columns[1]])
                strength = "strong positive" if corr > 0.7 else "moderate positive" if corr > 0.3 else "weak positive" if corr > 0 else "no" if corr == 0 else "weak negative" if corr > -0.3 else "moderate negative" if corr > -0.7 else "strong negative"
                answer = f"The correlation between '{columns[0]}' and '{columns[1]}' is {corr:.2f}, which indicates a {strength} correlation."

            elif question_info['type'] == 'filter':
                # Extract condition parts
                if question_info.get('conditions'):
                    # Process complex conditions
                    filter_expr = ""
                    for condition in question_info['conditions']:
                        if isinstance(condition, tuple):
                            if condition[0].startswith(('greater', 'more', 'higher', 'larger', 'bigger')):
                                filter_expr = f"{columns[0]} > {float(condition[1])}"
                            elif condition[0].startswith(('less', 'lower', 'smaller')):
                                filter_expr = f"{columns[0]} < {float(condition[1])}"
                            elif 'equal' in condition[0] or '=' in condition[0]:
                                filter_expr = f"{columns[0]} == {float(condition[1])}"
                            elif 'between' in condition[0] and len(condition) >= 2:
                                filter_expr = f"{columns[0]} > {float(condition[0])} and {columns[0]} < {float(condition[1])}"
                else:
                    # Try to infer condition from question
                    numbers = question_info.get('numbers', [])
                    if numbers and '>' in question:
                        filter_expr = f"{columns[0]} > {float(numbers[0])}"
                    elif numbers and '<' in question:
                        filter_expr = f"{columns[0]} < {float(numbers[0])}"
                    elif numbers and ('=' in question or 'equal' in question.lower()):
                        filter_expr = f"{columns[0]} == {float(numbers[0])}"
                    else:
                        # Default case
                        filter_expr = f"{columns[0]} > {self.df[columns[0]].mean()}"

                # Apply the filter
                filtered_df = self.df.query(filter_expr) if filter_expr else self.df
                if len(filtered_df) > 5:
                    answer = f"Found {len(filtered_df)} rows matching the condition. Here are the first 5:\n{filtered_df.head(5).to_string()}"
                else:
                    answer = f"Found {len(filtered_df)} rows matching the condition:\n{filtered_df.to_string()}"

            elif question_info['type'] == 'group' and len(columns) >= 2:
                # Determine which column to group by and which to aggregate
                numeric_col = None
                group_col = None

                for col in columns:
                    if pd.api.types.is_numeric_dtype(self.df[col].dtype):
                        numeric_col = col
                    else:
                        group_col = col

                # If we couldn't determine, assume first is group, second is value
                if numeric_col is None or group_col is None:
                    group_col = columns[0]
                    numeric_col = columns[1]

                # Perform groupby and aggregation
                result = self.df.groupby(group_col)[numeric_col].agg(['mean', 'sum', 'count']).reset_index()
                answer = f"Grouped by '{group_col}':\n{result.to_string()}"

            elif question_info['type'] == 'top' and columns:
                # Find top N values
                # Try to extract N from the question
                n = 5  # Default
                for num in question_info.get('numbers', []):
                    if int(float(num)) > 0:
                        n = int(float(num))
                        break

                top_values = self.df.nlargest(n, columns[0])
                answer = f"Top {n} values in '{columns[0]}':\n{top_values.to_string()}"

            elif question_info['type'] == 'bottom' and columns:
                # Find bottom N values
                # Try to extract N from the question
                n = 5  # Default
                for num in question_info.get('numbers', []):
                    if int(float(num)) > 0:
                        n = int(float(num))
                        break

                bottom_values = self.df.nsmallest(n, columns[0])
                answer = f"Bottom {n} values in '{columns[0]}':\n{bottom_values.to_string()}"

            elif question_info['type'] == 'distribution' and columns:
                # Describe the distribution
                desc = self.df[columns[0]].describe()
                answer = f"Distribution of '{columns[0]}':\n{desc.to_string()}"

                # Add visualization hint
                answer += "\n\nTo visualize this distribution, you can use:"
                answer += f"\n\nplt.figure(figsize=(10, 6))\nsns.histplot(data=df, x='{columns[0]}')\nplt.title('Distribution of {columns[0]}')\nplt.show()"

            elif question_info['type'] == 'trend' and len(columns) >= 2:
                # Determine which column is time and which is value
                time_col = None
                value_col = None

                for col in columns:
                    if self.column_descriptions[col].get('semantic_type') == 'date/time':
                        time_col = col
                    elif pd.api.types.is_numeric_dtype(self.df[col].dtype):
                        value_col = col

                # If we couldn't determine, assume first is time, second is value
                if time_col is None or value_col is None:
                    time_col = columns[0]
                    value_col = columns[1]

                # Create a simple description of the trend
                # Sort by time column
                trend_data = self.df.sort_values(time_col)
                first_value = trend_data[value_col].iloc[0]
                last_value = trend_data[value_col].iloc[-1]
                change = ((last_value - first_value) / first_value) * 100 if first_value != 0 else 0

                trend_direction = "increasing" if change > 5 else "decreasing" if change < -5 else "stable"

                answer = f"The trend of '{value_col}' over '{time_col}' is {trend_direction}. "
                answer += f"From {trend_data[time_col].iloc[0]} to {trend_data[time_col].iloc[-1]}, "
                answer += f"the value changed from {first_value:.2f} to {last_value:.2f} ({change:.1f}% change)."

                # Add visualization hint
                answer += "\n\nTo visualize this trend, you can use:"
                answer += f"\n\nplt.figure(figsize=(12, 6))\nplt.plot(df['{time_col}'], df['{value_col}'])\nplt.title('Trend of {value_col} over {time_col}')\nplt.xlabel('{time_col}')\nplt.ylabel('{value_col}')\nplt.grid(True)\nplt.show()"

            else:
                # Generic response for unrecognized question type
                answer = f"I couldn't fully understand your question about the data. Here's some basic information about the dataset:\n"
                answer += f"- Dataset has {self.data_summary['total_rows']} rows and {self.data_summary['total_columns']} columns\n"
                answer += f"- Columns: {', '.join(self.data_summary['column_names'])}\n"
                answer += "Please try asking a more specific question about the data."

        except Exception as e:
            # Handle any errors
            answer = f"Error processing your question: {str(e)}\n"
            answer += "Please try rephrasing your question or specify the column names more clearly."

        return answer

    def interactive_qa(self):
        """Start an interactive Q&A session with the user."""
        # Train the question classifier if not already trained
        if self.question_classifier is None:
            print("Training question classifier...")
            self.train_question_classifier()

        print("\n===== Excel Question Answering System =====")
        print(f"Loaded file: {self.file_path}")
        print(f"Dataset has {self.data_summary['total_rows']} rows and {self.data_summary['total_columns']} columns")
        print("Columns:", ", ".join(self.data_summary['column_names']))
        print("\nAsk questions about your data (type 'exit' to quit):")

        while True:
            question = input("\nYour question: ")

            if question.lower() in ['exit', 'quit', 'bye']:
                print("Goodbye!")
                break

            # Process the question and get the answer
            answer = self.answer_question(question)

            # Print the answer
            print("\nAnswer:")
            print(answer)

# Example usage
if __name__ == "__main__":
    # Replace with your Excel file path
    file_path = "/content/Online Retail.xlsx"

    # Create the QA system
    qa_system =ExcelQuestionAnswerer(file_path)

    # Start interactive session
    qa_system.interactive_qa()

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Successfully loaded data from /content/Online Retail.xlsx
Shape: (541909, 8)


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Training question classifier...
Classification Report:
              precision    recall  f1-score   support

 correlation       1.00      1.00      1.00         3
       count       0.00      0.00      0.00         1
distribution       1.00      1.00      1.00         1
      filter       0.67      1.00      0.80         2
         max       0.67      1.00      0.80         2
        mean       1.00      1.00      1.00         3
         min       1.00      0.50      0.67         2
         sum       1.00      1.00      1.00         1
         top       1.00      1.00      1.00         2
       trend       1.00      1.00      1.00         1

    accuracy                           0.89        18
   macro avg       0.83      0.85      0.83        18
weighted avg       0.87      0.89      0.86        18


===== Excel Question Answering System =====
Loaded file: /content/Online Retail.xlsx
Dataset has 541909 rows and 8 columns
Columns: InvoiceNo, StockCode, Description, Quantity, InvoiceD

# New Section